In [1]:
import pandas as pd
import random
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)
len(df)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


72947

In [5]:
df = df.dropna()
len(df)

70208

In [6]:
toxic_fields = [
    'toxic_score',
    'toxic_score.1',   
    'toxic_score.2',
    'toxic_score.3',    
    'toxic_score.4',    
]

In [7]:
def get_avg_toxicity(row):
    myList = [row[field] for field in toxic_fields]
    result = sum(myList)/20.0
    for field in toxic_fields:
        maxValue = max(myList)
        minValue = min(myList)
        if (maxValue - minValue > 1):
            return None
    return result

In [8]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [9]:
df = df.dropna()
len(df)

27994

In [10]:
def get_bucket(row, field):
    if (row[field] <= 0.2):
        return 0.0
    elif (row[field] < 0.4):
        return 0.2
    elif (row[field] < 0.6):
        return 0.4
    elif (row[field] < 0.8):
        return 0.6
    else:
        return 0.8

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)

In [12]:
myrange = [0.2, 0.4, 0.6, 0.8]
sampled_dfs = []
for ow in myrange:
    g = df[df['avg_toxic_window'] == ow]
    print (ow, len(g))
    sampled_dfs.append(g.sample(10))

0.2 1105
0.4 356
0.6 141
0.8 143


In [13]:
final_df = pd.concat(sampled_dfs)

In [14]:
def get_random_date():
    random_number_of_days = random.randrange(40)
    random_date = datetime.date(2021, 10, 1) + datetime.timedelta(days=random_number_of_days)
    return random_date

In [15]:
def assignPeripherals(final_df):
    final_df = final_df[['comment', 'avg_toxic_score', 'comment_ID']]
    
    # Set username
    usernameDf = pd.read_csv('usernames.csv')
    usernames = set(list(usernameDf['username']))
    final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
    
    # Set full name
    nameDf = pd.read_csv('random_names.csv')
    names = set(list(nameDf['name']))
    final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)    
    
    # Set integer fields
    final_df['num_likes'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_comments'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_retweets'] = np.random.randint(1, 100, final_df.shape[0])        
    
    # Set publication date
    final_df['pub_date'] = df.apply(lambda row: get_random_date(), axis=1)
    
    return final_df

In [16]:
final_df = assignPeripherals(final_df)
final_df.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
68411,Don't you have problems in your own country to solve ? Get a life.,0.30,4822_12,meaningfully_usable_airtime,Lyndon Barrett,85,11,50,2021-10-30
17431,I’m betting Acosta is a pedophile himself.,0.20,746_16,warm_roadblock,Sophia Grant,46,51,63,2021-10-20
28424,I’m sure you were just “trolling” do something better with your life,0.20,2876_8,doting_lineage,Ada Jones,55,47,46,2021-10-25
21385,"Your whole life has been a handout, Tammy.",0.35,516_7,bravelypracticalchina,Amber Kelly,41,62,70,2021-11-09
22120,Good morning! GOP = Gross Old Perverts,0.35,1331_11,thefrizzyforgiveness,Chloe Williams,81,98,5,2021-10-13


In [17]:
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.to_csv('main_comments.csv', index=False)

In [18]:
nontoxic_df = df.query('avg_toxic_window == 0.0')

In [19]:
nontoxic_sample = nontoxic_df.sample(30)

In [20]:
nontoxic_sample = assignPeripherals(nontoxic_sample)
nontoxic_sample.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
4872,The worst twitter take ever,0.05,623_21,thelyricalmirth,Ashton Campbell,67,12,22,2021-10-24
42142,The d****** part really hit close to home 😔😔,0.10,2642_21,intrigued_tights,Arianna Evans,61,98,87,2021-10-26
63279,weed is satanic apparently,0.05,4627_22,exciting_self-control,Garry Walker,29,21,5,2021-11-05
58236,We all ugly to somebody. ☠️,0.15,4784_5,thefrizzyforgiveness,Owen Kelly,43,5,42,2021-11-05
22658,"Honestly , the french server is very supported . Ive lost my account bc I forgot my pw bc im dumb 🤪🤪. But friend mE; målibu",0.00,3544_7,warm_roadblock,Haris Cole,25,21,58,2021-10-13


In [21]:
nontoxic_sample.to_csv('nontoxic_comments.csv', index=False)